**Loading Libraries**

In [ ]:
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np

**Loading Datasets**

In [ ]:
#Loading Raw Datasets
master = pd.read_csv("\\Users\\kevdr\\OneDrive\\Desktop\\Master\\Köln\\Business Analytics\\Capstone\\Data\\master.csv")
transfer = pd.read_csv("\\Users\\kevdr\\OneDrive\\Desktop\\Master\\Köln\\Business Analytics\\Capstone\\Data\\transfer.csv")
marketValue = pd.read_csv("\\Users\\kevdr\\OneDrive\\Desktop\\Master\\Köln\\Business Analytics\\Capstone\\Data\\marketValuebefore.csv")
match = pd.read_csv("\\Users\\kevdr\\OneDrive\\Desktop\\Master\\Köln\\Business Analytics\\Capstone\\Data\\matchfinal_fixed.csv")
lineup = pd.read_csv("\\Users\\kevdr\\OneDrive\\Desktop\\Master\\Köln\\Business Analytics\\Capstone\\Data\\lineupfinal_fixed.csv")
team_value = pd.read_csv("\\Users\\kevdr\\OneDrive\\Desktop\\Master\\Köln\\Business Analytics\\Capstone\\Data\\team_market_value.csv")

**Shape of Datasets**

In [ ]:
# Calculatin columns and rows for each dataframe
master_rows, master_cols = master.shape
transfer_rows, transfer_cols = transfer.shape
marketValue_rows, marketValue_cols = marketValue.shape
matchfinal_rows, matchfinal_cols = match.shape
lineupfinal_rows, lineupfinal_cols = lineup.shape

# Print the number of rows and columns for each dataframe
print(f"Masterdata Data Set: {master_rows} rows and {master_cols} columns")
print(f"Transfer Data Set: {transfer_rows} rows and {transfer_cols} columns")
print(f"Market Value Data Set: {marketValue_rows} rows and {marketValue_cols} columns")
print(f"Match Final Data Set: {matchfinal_rows} rows and {matchfinal_cols} columns")
print(f"Lineup Final Data Set: {lineupfinal_rows} rows and {lineupfinal_cols} columns")

In [ ]:
print(tabulate(master.head(3), headers='keys', tablefmt='pretty'))
print(tabulate(transfer.head(3), headers='keys', tablefmt='pretty'))
print(tabulate(marketValue.head(3), headers='keys', tablefmt='pretty'))
print(tabulate(match.head(3), headers='keys', tablefmt='pretty'))
print(tabulate(lineup.head(3), headers='keys', tablefmt='pretty'))

**Pre-Processing**

Standardizing Column Names

In [ ]:
# Standardizing Column Name for comeptition, season, playerid for every dataset
master.rename(columns={'playerId': 'playerId'}, inplace=True)
transfer.rename(columns={'playerId': 'playerId'}, inplace=True)
marketValue.rename(columns={'playerId': 'playerId'}, inplace=True)
lineup.rename(columns={'player_id': 'playerId'}, inplace=True)

#MatchId
match.rename(columns={'match_id': 'matchId'}, inplace=True)
lineup.rename(columns={'match_id': 'matchId'}, inplace=True)

#Competition Id
lineup.rename(columns={'competition_id': 'competitionId'}, inplace=True)

#Change name of austrian bundesliga
transfer.loc[
    (transfer['from_competition_competition_name'] == 'Bundesliga') & 
    (transfer['from_competition_competition_area'] == 'Austria'),
    'from_competition_competition_name'
] = 'Austrian Bundesliga'

transfer.loc[
    (transfer['to_competition_competition_name'] == 'Bundesliga') & 
    (transfer['to_competition_competition_area'] == 'Austria'),
    'to_competition_competition_name'
] = 'Austrian Bundesliga'

print("Unique playerIds in transfer:", transfer['playerId'].nunique())

# Count of rows  in transfer after filtering
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

Converting Data Types

In [ ]:
# First, normalize the playerId across all dataframes by converting to int and then to str
master['playerId'] = master['playerId'].astype(int)
transfer['playerId'] = transfer['playerId'].astype(int)
marketValue['playerId'] = marketValue['playerId'].astype(int)
lineup['playerId'] = lineup['playerId'].astype(int)
lineup['matchId'] = lineup['matchId'].astype(str)
match['matchId'] = match['matchId'].astype(str)

# Converting to Integer
master['yearOfBirth'] = master['yearOfBirth'].astype('Int64')
master['height'] = master['height'].astype('Int64')
master['marketValue'] = master['marketValue'].astype('Int64')
marketValue['marketvalue'] = marketValue['marketvalue'].astype('Int64')
transfer['fee'] = transfer['fee'].astype('Int64')

#Converting string
lineup["competitionId"] = lineup["competitionId"].astype(str)

#Converting string in Season
lineup['season'] = lineup['season'].astype(str)
transfer['season'] = transfer['season'].astype(str)
match['season'] = match['season'].astype(str)
team_value['season_id'] = team_value['season_id'].astype(str)  # Ensure season_id is string for merging


# Convert 'scoreHome' and 'scoreAway' columns to numeric values;
# if conversion fails (e.g., due to non-numeric strings), replace with NaN
match['scoreHome'] = pd.to_numeric(match['scoreHome'], errors='coerce')
match['scoreAway'] = pd.to_numeric(match['scoreAway'], errors='coerce')

# then change it into 'Int64'
match['scoreHome'] = match['scoreHome'].astype('Int64')
match['scoreAway'] = match['scoreAway'].astype('Int64')
lineup['home_team'] = lineup['home_team'].astype(bool)

Checking Unique PlayerIds in Transfer, Master, LineUp

In [ ]:
# checking Unique PlayerIds 
print("playerIds in transfer:", transfer['playerId'].nunique())
print("playerIds in master:", master['playerId'].nunique())
print("playerIds in lineup:", lineup['playerId'].nunique())

# checking itnersection of playerIds
transfer_ids = set(transfer['playerId'])
common_with_master = transfer_ids.intersection(set(master['playerId']))
common_with_lineup = transfer_ids.intersection(set(lineup['playerId']))
common_lineup_master = set(lineup['playerId']).intersection(set(master['playerId']))

print("playerIds in transfer ∩ master:", len(common_with_master))
print("playerIds in transfer ∩ lineup:", len(common_with_lineup))
print("playerIds in lineup ∩ master:", len(common_lineup_master))


common_all = transfer_ids.intersection(set(master['playerId']), set(lineup['playerId']))
print("playerIds in transfer ∩ master ∩ lineup:", len(common_all))

Dropping / Adding Columns

In [ ]:
# count of rows of match and lineup
print(f"Count of Rows in match before filtering: {len(match)}")
print(f"Count of Rows in lineup before filtering: {len(lineup)}")

#Dropping columns
transfer = transfer.drop(columns=['internalId', 'externalId', 'playerInternalId','feePounds'])
master = master.drop(columns=['secondNationality', 'secondaryPosition2'])	

#perf = perf.drop(columns=['index'])
match = match.drop(columns = ['updatedAt', 'round', 'deleted', 'version'])

#dropping duplicates for lineup data when playerid and match id is same
lineup = lineup.drop_duplicates(subset=['playerId', 'matchId'], keep='first')

#dropping duplicates
master = master.drop_duplicates(keep='last')

#perf = perf.drop_duplicates(keep='last')
transfer = transfer.drop_duplicates(keep='last')

#injury = injury.drop_duplicates(keep='last')
marketValue = marketValue.drop_duplicates(keep='last')

#comp = comp.drop_duplicates(keep='last')
match = match.drop_duplicates(subset =['matchId'], keep='first')

## Merge First and Last Name into Full Name
master['playerName'] = master['firstName'] + ' ' + master['lastName']
master = master.drop(columns=['firstName', 'lastName'])

# Reorder columns to put playerName right after playerID
cols = master.columns.tolist()
playerid_index = cols.index('playerId')
cols.insert(playerid_index + 1, cols.pop(cols.index('playerName')))
master = master[cols]

#cols = perf.columns.tolist()
playerid_index = cols.index('playerId')
cols.insert(playerid_index + 1, cols.pop(cols.index('playerName')))
print("Unique playerIds in transfer:", transfer['playerId'].nunique())

# Count of rows  in transfer after filtering
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

## Add transferage
# Merge transfer dataset with masterdata to match playerId
merged_transfer = pd.merge(transfer, master[['playerId', 'height', 'mainPosition', 'positionGroup', 'foot', 'dateOfBirth', 'nationality']], on='playerId', how='left')

# Calculate the transfer age: difference between transfer date and dateOfBirth
merged_transfer['transferAge'] = pd.to_datetime(merged_transfer['date']).dt.year - pd.to_datetime(merged_transfer['dateOfBirth']).dt.year

# If needed, drop the 'dateOfBirth' column after the calculation
merged_transfer = merged_transfer.drop(columns=['dateOfBirth'])

# If you want to update the original 'transfer' DataFrame
transfer = merged_transfer
transfer['transferAge'] = transfer['transferAge'].astype('Int64')

## Reorder columns 
# Get current list of columns
cols = transfer.columns.tolist()

# Find the index of the 'season' column
season_index = cols.index('playerName')

# Remove 'transferAge' from the list
cols.remove('transferAge')

# Insert them right after 'season'
cols.insert(season_index + 1, 'transferAge')

# Reorder the DataFrame
transfer = transfer[cols]
print(tabulate(transfer.head(20), headers='keys', tablefmt='pretty'))
print("Unique playerIds in transfer:", transfer['playerId'].nunique())

# Count of rows  in transfer after filtering
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

# count of rows of match and lineup
print(f"Count of Rows in match after filtering: {len(match)}")
print(f"Count of Rows in lineup after filtering: {len(lineup)}")

Filtering transfer for selecting last transfer

In [ ]:
# Filtering transfer for selecting last transfer because for multiple transfer or loans in one season
# Convert the 'date' column to datetime format
transfer['date'] = pd.to_datetime(transfer['date'])

# Sort the dataframe by 'playerid' and 'date'
transfer = transfer.sort_values(by=['playerId', 'date'])

# Define a function to filter transfer within a 6-month window
def filter_transfer(group):
    filtered = []
    for i in range(len(group)):
        if i == len(group) - 1 or (group.iloc[i + 1]['date'] - group.iloc[i]['date']).days > 182:
            filtered.append(group.iloc[i])
    return pd.DataFrame(filtered)

# Group by 'playerid' and apply the filtering function
transfer = transfer.groupby('playerId', group_keys=False).apply(filter_transfer)
transfer.head()
print("Unique playerIds in transfer:", transfer['playerId'].nunique())

# Count of rows  in transfer after filtering
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

Dropping extra Match Id in Match Dataset

In [ ]:
# checking uniqer entries of playerId
print("Unique match_id values in lineup und match:")
print(lineup['matchId'].nunique())
print(match['matchId'].nunique())

# common entries in matchId
common_matches = pd.merge(lineup, match, on='matchId', how='inner')
print("Common entries in match_id:")
print(common_matches['matchId'].nunique())

# Showing matchids of non common matches in dataset match
non_common_matches = match[~match['matchId'].isin(common_matches['matchId'])]

# Dropping non common matches Match Ids in dataset Match
# Dropping U16 U17 Matches
match = match[match['matchId'].isin(common_matches['matchId'])]
print("Unique entries in match_id after dropping:")
print(match['matchId'].nunique())

**Performance Data**

Dropping Na Values

In [ ]:
# number of na valuex in transfe rdatset in competition_competition_end
print("Anzahl der NaN-Werte in 'to_competition_competition_end_date':", transfer['to_competition_competition_end_date'].isna().sum())

# umber of na valuex in transfe rdatset in competition_competition_end groupebd by comeptition Id
na_counts = transfer.groupby('to_competition')['to_competition_competition_end_date'].apply(lambda x: x.isna().sum())
print("Anzahl der NaN-Werte in 'to_competition_competition_end_date' gruppiert nach 'to_competition':")

#print sorted by count
na_counts = na_counts.sort_values(ascending=False)
print(na_counts)

#dropping Nas in ciompetition end date
transfer = transfer.dropna(subset=['to_competition_competition_end_date'])

# number of na values in transfer dataset in date
print("Anzahl der NaN-Werte in 'date':", transfer['date'].isna().sum())

#dropping nan values in date
transfer = transfer.dropna(subset=['date'])
print("Unique playerIds in transfer:", transfer['playerId'].nunique())

# Count of rows  in transfer after filtering
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

Adding Performance Data before and after

In [ ]:
# === Add match date to lineup ===
lineup = lineup.merge(match[['matchId', 'date']], on='matchId', how='left')
lineup['date'] = pd.to_datetime(lineup['date']).dt.strftime('%Y-%m-%d')
lineup['date'] = pd.to_datetime(lineup['date']).dt.tz_localize(None)

# === Prepare transfer data ===
transfer['to_competition_competition_end_date'] = pd.to_datetime(transfer['to_competition_competition_end_date'])
transfer['to_competition_competition_end_date'] = transfer['to_competition_competition_end_date'].dt.strftime('%m-%d')
transfer['date'] = pd.to_datetime(transfer['date']).dt.tz_localize(None)

# === Calculate real end date based on transfer year ===
def compute_end_date(row):
    transfer_year = row['date'].year
    end_date = pd.to_datetime(f"{transfer_year}-{row['to_competition_competition_end_date']}")
    if end_date <= row['date']:
        end_date = end_date.replace(year=transfer_year + 1)
    return end_date.strftime('%Y-%m-%d')

transfer['to_competition_competition_end_date'] = transfer.apply(compute_end_date, axis=1)

transfer['start_date'] = pd.to_datetime(transfer['date'])
transfer['end_date'] = pd.to_datetime(transfer['to_competition_competition_end_date'])

# === Filter valid lineups ===
merged = lineup.merge(
    transfer[['playerId', 'start_date', 'end_date']],
    on='playerId',
    how='inner'
)

valid_rows = merged[
    (merged['date'] >= merged['start_date']) &
    (merged['date'] <= merged['end_date'])
]

# === Merge match scores only once ===
valid_rows = valid_rows.merge(match[['matchId', 'scoreHome', 'scoreAway']], on='matchId', how='left')

# === Clean sheet calculation ===
def is_clean_sheet(row):
    if row['home_team']:
        return row['scoreAway'] == 0
    else:
        return row['scoreHome'] == 0

valid_rows['minutesPlayed'] = pd.to_numeric(valid_rows['minutesPlayed'], errors='coerce').fillna(0).astype(int)
valid_rows['clean_sheet_raw'] = valid_rows.apply(is_clean_sheet, axis=1)
valid_rows['clean_sheet_raw'] = valid_rows['clean_sheet_raw'].fillna(False).astype(bool)
valid_rows['clean_sheet'] = valid_rows.apply(
    lambda row: int(row['clean_sheet_raw'] and row['minutesPlayed'] >= 5),
    axis=1
)

# === Group by player/period and aggregate ===
group_cols = ['playerId', 'start_date', 'end_date']
minutes_summary = (
    valid_rows
    .groupby(group_cols)
    .agg(
        total_minutes_played=('minutesPlayed', 'sum'),
        total_games=('date', 'count'),
        goals_scored=('goals', 'sum'),
        assists=('assists', 'sum'),
        clean_sheets=('clean_sheet', 'sum')
    )
    .reset_index()
)
minutes_summary['total_possible_minutes'] = minutes_summary['total_games'] * 90

# === Merge aggregated stats back to transfer ===
transfer = transfer.merge(minutes_summary, on=group_cols, how='left')
print("Unique playerIds in transfer:", transfer['playerId'].nunique())
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

# === PREVIOUS SEASON PERFORMANCE ===

transfer_prev = transfer.copy()
transfer_prev['start_date'] = transfer_prev['start_date'] - pd.DateOffset(years=1)
transfer_prev['end_date'] = transfer_prev['end_date'] - pd.DateOffset(years=1)

merged_prev = lineup.merge(
    transfer_prev[['playerId', 'start_date', 'end_date']],
    on='playerId',
    how='inner'
)
valid_rows_prev = merged_prev[
    (merged_prev['date'] >= merged_prev['start_date']) &
    (merged_prev['date'] <= merged_prev['end_date'])
]

# Merge scores once
valid_rows_prev = valid_rows_prev.merge(match[['matchId', 'scoreHome', 'scoreAway']], on='matchId', how='left')

valid_rows_prev['minutesPlayed'] = pd.to_numeric(valid_rows_prev['minutesPlayed'], errors='coerce').fillna(0).astype(int)
valid_rows_prev['clean_sheet_raw'] = valid_rows_prev.apply(is_clean_sheet, axis=1)
valid_rows_prev['clean_sheet_raw'] = valid_rows_prev['clean_sheet_raw'].fillna(False).astype(bool)
valid_rows_prev['clean_sheet'] = valid_rows_prev.apply(
    lambda row: int(row['clean_sheet_raw'] and row['minutesPlayed'] >= 5),
    axis=1
)

minutes_summary_prev = (
    valid_rows_prev
    .groupby(['playerId', 'start_date', 'end_date'])
    .agg(
        prev_total_minutes_played=('minutesPlayed', 'sum'),
        prev_total_games=('date', 'count'),
        goals_scored_before=('goals', 'sum'),
        assists_before=('assists', 'sum'),
        clean_sheets_before=('clean_sheet', 'sum')
    )
    .reset_index()
)
minutes_summary_prev['prev_total_possible_minutes'] = minutes_summary_prev['prev_total_games'] * 90

transfer_prev = transfer_prev.merge(
    minutes_summary_prev,
    on=['playerId', 'start_date', 'end_date'],
    how='left'
)

transfer_prev['start_date'] = transfer_prev['start_date'] + pd.DateOffset(years=1)
transfer_prev['end_date'] = transfer_prev['end_date'] + pd.DateOffset(years=1)

transfer = transfer.merge(
    transfer_prev[['playerId', 'start_date', 'end_date',
                   'prev_total_minutes_played', 'prev_total_games', 'prev_total_possible_minutes',
                   'goals_scored_before', 'assists_before', 'clean_sheets_before']],
    on=['playerId', 'start_date', 'end_date'],
    how='left'
)

print("Transfer data enriched with previous season")
print(transfer[['playerId', 'start_date', 'total_minutes_played', 'prev_total_minutes_played']].head())

Calculating percentage played

In [ ]:
#Calculation for Percentage Played before and after
transfer['percentage_played'] = (transfer['total_minutes_played'] / transfer['total_possible_minutes']) * 100
transfer["percentage_played"] = transfer["percentage_played"].round(2)
transfer['percentage_played_before'] = (transfer['prev_total_minutes_played'] / transfer['prev_total_possible_minutes']) * 100
transfer["percentage_played_before"] = transfer["percentage_played_before"].round(2)

# range of values for percentage played in league_perf
print("Range of values for percentage played in league_perf:")
print(transfer["percentage_played"].describe())

# range of values for percentage played in league_perf
print("Range of values for percentage played before in league_perf:")
print(transfer["percentage_played_before"].describe())


# plot of distribution of percentage played in transfer dataset
plt.figure(figsize=(10, 6))
plt.hist(transfer['percentage_played'], bins=30, color='blue', alpha=0.7)
plt.title('Distribution of Percentage Played in Transfer Dataset')
plt.xlabel('Percentage Played')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

plt.figure(figsize=(10, 6))
plt.hist(transfer['percentage_played_before'], bins=30, color='blue', alpha=0.7)
plt.title('Distribution of Percentage Played befpre in Transfer Dataset')
plt.xlabel('Percentage Played')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

Dropping unrealistic calculations of the league in malta and dropping transfer with multiple entries

In [ ]:
# showing rows with more than 100 percentage played
print("Rows with more than 100 percentage played:")
nonsense = transfer[transfer["percentage_played"] > 100]
print(nonsense)

# dropping unrealistic percentage played values
transfer = transfer[(transfer["percentage_played"] <= 100) | (transfer["percentage_played"].isna())]
print("Unique playerIds in transfer:", transfer['playerId'].nunique())

# Count of rows  in transfer after filtering
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

# counting how many players have same competitionId playerId and season
transfer['count'] = transfer.groupby(['playerId', 'to_competition', 'season'])['playerId'].transform('count')

#showing number
print(transfer['count'].value_counts())

#dropping values with count > 1
transfer = transfer[transfer['count'] == 1]

# Count of rows  in transfer after filtering
print("Unique playerIds in transfer:", transfer['playerId'].nunique())
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

Adding closest marketvalue

In [ ]:
# Checking if both date are same structure
marketValue['date'] = pd.to_datetime(marketValue['date']).dt.tz_localize(None)
transfer['date'] = pd.to_datetime(transfer['date']).dt.tz_localize(None)

# Function to find the closest market value before the transfer date
def find_closest_marketvalue(row):
    player_id = row['playerId']
    transfer_date = row['date']
    
    # Filter for matching player
    matching = marketValue[marketValue['playerId'] == player_id]
    
    # Only keep dates before the transfer date
    matching = matching[matching['date'] <= transfer_date]
    
    if matching.empty:
        return None

    # Calculate time delta (difference in days)
    matching['timedelta'] = (transfer_date - matching['date'])
    
    # Get the row with the smallest timedelta
    closest = matching.loc[matching['timedelta'].idxmin()]
    return closest['marketvalue']

# Apply the function to each transfer row
transfer['marketvalue_closest'] = transfer.apply(find_closest_marketvalue, axis=1)

In [ ]:
print("Unique playerIds in transfer:", transfer['playerId'].nunique())
# Count of rows  in transfer after filtering
print(f"Count of Rows in transfer after filtering: {len(transfer)}")

Adding Market Value of Team/League

In [ ]:
# Keep only unique and valid entries per team and season
team_value = team_value.drop_duplicates(subset=['externalId', 'season_id'], keep='last')

# Calculate the smallest valid market value (> 0)
min_mv = team_value.loc[team_value['marketValue'] > 0, 'marketValue'].min()

# Replace 0 market values with the smallest valid market value
team_value['marketValue'] = team_value['marketValue'].replace(0, min_mv)

# Ensure season columns are integers
team_value['season_id'] = team_value['season_id'].astype(int)
transfer['season'] = transfer['season'].astype(int)

# Fallback function to retrieve closest market value by season if exact match is missing
def get_closest_market_value(team_id, season, df_teamval):
    subset = df_teamval[df_teamval['externalId'] == team_id]
    if subset.empty:
        return None
    subset = subset.copy()
    subset['season_diff'] = (subset['season_id'] - season).abs()
    return subset.sort_values('season_diff').iloc[0]['marketValue']

# Copy of transfer data to avoid modifying the original with new columns
transfer_result = transfer.copy()

### FROM-TEAM MARKET VALUE
# Temporary dataframe with only necessary columns for from team
temp_from = team_value[['externalId', 'season_id', 'marketValue']].rename(columns={
    'externalId': 'from_teamId',
    'season_id': 'season',
    'marketValue': 'fromTeam_marketValue'
})

# Merge transfer with from team market value
transfer_result = transfer_result.merge(temp_from, on=['from_teamId', 'season'], how='left')

# Fallback for missing from team market value
transfer_result['fromTeam_marketValue'] = transfer_result.apply(
    lambda row: row['fromTeam_marketValue'] if pd.notna(row['fromTeam_marketValue'])
    else get_closest_market_value(row['from_teamId'], row['season'], team_value),
    axis=1
)

### TO-TEAM MARKET VALUE
# Temporary dataframe with only necessary columns for to team
temp_to = team_value[['externalId', 'season_id', 'marketValue']].rename(columns={
    'externalId': 'to_teamId',
    'season_id': 'season',
    'marketValue': 'toTeam_marketValue'
})

# Merge transfer with to team market value
transfer_result = transfer_result.merge(temp_to, on=['to_teamId', 'season'], how='left')

# Fallback for missing to team market value
transfer_result['toTeam_marketValue'] = transfer_result.apply(
    lambda row: row['toTeam_marketValue'] if pd.notna(row['toTeam_marketValue'])
    else get_closest_market_value(row['to_teamId'], row['season'], team_value),
    axis=1
)

# Final cleanup: keep only original columns + the two market value columns
columns_to_keep = transfer.columns.tolist() + ['fromTeam_marketValue', 'toTeam_marketValue']
transfer_result = transfer_result[columns_to_keep]

# Assign final result back to transfer dataframe
transfer = transfer_result

### === Calculate ratio between team market values ===
transfer['team_market_value_relation'] = (
    transfer['toTeam_marketValue'] / transfer['fromTeam_marketValue']
).round(2)

Adding Goals, assist, clean sheet, scorers and binary threshold

In [ ]:
# Copying for 
final_dataset = transfer.copy()

#Adding binary threshold for percentage played when bigger equal to 50
final_dataset['success'] = final_dataset['percentage_played'].apply(lambda x: 1 if x >= 50 else 0) 

#Adding binary threshold for percentage played when bigger equal to 50
final_dataset['success_before'] = final_dataset['percentage_played_before'].apply(lambda x: 1 if x >= 50 else 0)          

#sort final dataset by playerId, season, competitionId
final_dataset = final_dataset.sort_values(by=['playerId', 'season'])

# === Feature: scorer_before (before transfer) ===
# Total of goals and assists before the transfer
final_dataset["scorer_before"] = final_dataset["goals_scored_before"] + final_dataset["assists_before"]

#count rows and columns of final dataset
final_dataset_rows, final_dataset_cols = final_dataset.shape
print(f"Final Data Set: {final_dataset_rows} rows and {final_dataset_cols} columns")

Adding fee to value ratio, foreign transfers, value age product and value per age

In [ ]:
# === Feature 1: fee_to_value_ratio ===
# Calculate only when both fee and market value are valid (not NaN and not zero)
fee = final_dataset["fee"]
marketvalue = final_dataset["marketvalue_closest"]

# Initialize result array with NaNs
fee_to_value_ratio = np.full(len(final_dataset), np.nan)

# Define a mask for valid rows
mask = (
    fee.notna() &
    marketvalue.notna() &
    (fee != 0) &
    (marketvalue != 0)
)

# Compute the ratio only for valid entries
fee_to_value_ratio[mask] = fee[mask].values / marketvalue[mask].values

# Assign the result to the dataframe
final_dataset["fee_to_value_ratio"] = fee_to_value_ratio

# === Feature: foreign_transfer ===
# Set to 1 if player moved to a country different from the previous league AND it's not the player's nationality
final_dataset["foreign_transfer"] = np.where(
    (final_dataset["from_competition_competition_area"] != final_dataset["to_competition_competition_area"]) & 
    (final_dataset["to_competition_competition_area"] != final_dataset["nationality"]),
    1,
    0
)

# value age product
final_dataset['value_age_product'] = final_dataset['marketvalue_closest'] * final_dataset['transferAge']

# value per age
final_dataset['value_per_age'] = final_dataset['marketvalue_closest'] / final_dataset['transferAge']

Adding Joker Feature

In [ ]:
# Replace zero percentage_played to avoid division by zero
final_dataset['percentage_played_before'] = final_dataset['percentage_played_before'].replace(0, 0.000001)

# Compute joker_ratio (goals per % played)
final_dataset['joker_ratio'] = final_dataset['goals_scored_before'] / final_dataset['percentage_played_before']

# Set joker_ratio and was_joker to NaN if percentage_played_before is NaN
mask_na = final_dataset['percentage_played_before'].isna()
final_dataset.loc[mask_na, ['joker_ratio', 'was_joker']] = np.nan

# Threshold for being a joker: 90th percentile
threshold = final_dataset['joker_ratio'].quantile(0.90)

# Flag joker players (only if percentage_played_before is not NaN)
final_dataset['was_joker'] = (
    (final_dataset['joker_ratio'] > threshold) & 
    (final_dataset['percentage_played_before'] < 50)
)

# Optional: overwrite again to enforce NaNs for invalid rows (after flagging)
final_dataset.loc[mask_na, ['joker_ratio', 'was_joker']] = np.nan

# Sort and display top joker players
joker_players = final_dataset[final_dataset['was_joker'].fillna(False)].sort_values(by='joker_ratio', ascending=False)

# Columns to show
cols_to_show = ['playerName', 'goals_scored_before', 'percentage_played_before', 'joker_ratio', 'was_joker']
print(tabulate(joker_players[cols_to_show].head(10), headers='keys', tablefmt='fancy_grid', floatfmt=".2f"))

Grouping scorer, clean sheets

In [ ]:
# clean_sheets grouped with bins (0-2, 2-5, 5-10, 10-15, 15-20, 20+)
bins_clean_sheets = [0, 2, 5, 10, 15, np.inf]
labels_clean_sheets = ['0-1', '2-4', '5-9', '10-14', '15+']
final_dataset['clean_sheets_before_grouped'] = pd.cut(final_dataset['clean_sheets_before'], bins=bins_clean_sheets, labels=labels_clean_sheets, right=False)

# Grouping Scorers without defenders and goalkeepers 
def bin_scorer(row):
    scorer = row["scorer_before"]
    position = str(row["mainPosition"]).lower()

    if pd.isna(scorer):
        return np.nan  # wichtig: NaN beibehalten

    if "goalkeeper" in position or "defender" in position:
        return "defender/goalkeeper"
    else:
        if scorer <= 3:
            return "0-3"
        elif scorer <= 6:
            return "4-6"
        elif scorer <= 10:
            return "7-10"
        elif scorer <= 15:
            return "11-15"
        elif scorer <= 20:
            return "16-20"
        elif scorer <= 30:
            return "21-30"
        else:
            return "30+"

final_dataset["scorer_before_grouped_category"] = final_dataset.apply(bin_scorer, axis=1)

final_dataset["scorer_before_grouped_category"] = pd.Categorical(
    final_dataset["scorer_before_grouped_category"],
    categories=[
        "defender/goalkeeper", "0-3", "4-6", "7-10", "11-15",
        "16-20", "21-30", "30+"
    ],
    ordered=True
)

# scorer only for midfielder and attackers
# Kopiere scorer_before in neue Spalte
final_dataset["scorer_before_new"] = final_dataset["scorer_before"]

# Wenn position goalkeeper oder defender → scorer = 0, aber nur falls scorer vorher nicht NaN war
mask = final_dataset["positionGroup"].str.lower().str.contains("goalkeeper|defender", na=False) 

final_dataset.loc[mask, "scorer_before_new"] = np.nan

**Cleaning columns and reorder**

In [ ]:
new_order = [
    # Player info
    'playerId', 'playerName', 'height', 'mainPosition', 'positionGroup', 'foot', 'nationality',

    # Transfer info
    'date', 'season', 'transferAge','isLoan', 'wasLoan', 'fee', 'marketvalue_closest', 'fee_to_value_ratio', 'foreign_transfer', 'value_age_product', 'value_per_age',

    # From team
    'from_team', 'from_teamId', 'from_competition',
    'from_competition_competition_name', 'from_competition_competition_area',
    'from_competition_competition_age_category', 'from_competition_competition_level',
    'from_competition_competition_association', 'fromTeam_marketValue',

    # To team
    'to_team', 'to_teamId', 'to_competition',
    'to_competition_competition_name', 'to_competition_competition_area',
    'to_competition_competition_age_category', 'to_competition_competition_level',
    'to_competition_competition_association', 'toTeam_marketValue', 'team_market_value_relation',

    # Playing time current season
    'start_date', 'end_date', 'total_minutes_played', 'total_games', 'total_possible_minutes',
    'percentage_played',

    # Playing time previous season
    'prev_total_minutes_played', 'prev_total_games', 'prev_total_possible_minutes',
    'percentage_played_before', 'was_joker',

    # Performance previous season
    'goals_scored_before','assists_before', 'scorer_before', 'scorer_before_new', 'scorer_before_grouped_category', 'clean_sheets_before', 'clean_sheets_before_grouped',

    # Success metrics
    'success', 'success_before'
]
final_dataset = final_dataset[new_order]

final_dataset_rows, final_dataset_cols = final_dataset.shape
print(f"Final Data Set: {final_dataset_rows} rows and {final_dataset_cols} columns")

Dropping NAs in Final Dataset

In [ ]:
# ---- Define the list of columns to check ----
cols_to_check = [
    # Current season performance
    'total_minutes_played', 'total_games', 'total_possible_minutes',
    'percentage_played',

    # Previous season playing time
    'prev_total_minutes_played', 'prev_total_games', 'prev_total_possible_minutes',
    'percentage_played_before', 'was_joker',

    # Previous season performance
    'goals_scored_before', 'assists_before', 'scorer_before',
    'scorer_before_new', 'scorer_before_grouped_category',
    'clean_sheets_before', 'clean_sheets_before_grouped'
]

# ---- Drop rows where ALL of these columns are NaN ----
before = len(final_dataset)  # or your DataFrame name
final_dataset.dropna(subset=cols_to_check, how='all', inplace=True)
after = len(final_dataset)

print(f"Rows dropped because all {len(cols_to_check)} columns were NaN: {before - after}")
print(f"Remaining rows: {after}")

**Saving Final Dataset**

In [ ]:
final_dataset.to_csv("\\Users\\kevdr\\OneDrive\\Desktop\\Master\\Köln\\Business Analytics\\Capstone\\Data\\final_dataset.csv", index=False)

**Data Distribution**

In [ ]:
# Load the dataset
df = pd.read_csv('Data/final_dataset.csv')

# Important features to visualize
important_features = [
    'height', 'transferAge', 'isLoan', 'wasLoan', 'was_joker', 'foreign_transfer',
    'percentage_played_before', 'percentage_played',
    'scorer_before_grouped_category', 'clean_sheets_before_grouped',
    'fromTeam_marketValue', 'toTeam_marketValue', 'marketvalue_closest',
    'from_competition_competition_level', 'to_competition_competition_level',
    'foot', 'mainPosition', 'positionGroup',
    'from_competition_competition_area', 'to_competition_competition_area',
    'value_per_age', 'value_age_product', 'team_market_value_relation'
]

# Features to apply log transformation for better distribution visualization
log_transform_features = [
    'fromTeam_marketValue', 'toTeam_marketValue', 'marketvalue_closest',
    'value_per_age', 'value_age_product', 'team_market_value_relation'
]

# Binary features (0 or 1)
binary_features = ['isLoan', 'wasLoan', 'was_joker', 'foreign_transfer']

# Categorical features that are integers (levels 1-4)
categorical_integer_features = ['from_competition_competition_level', 'to_competition_competition_level']

# Categorical string-based features
categorical_features = [
    'foot', 'mainPosition', 'positionGroup',
    'from_competition_competition_area', 'to_competition_competition_area',
    'scorer_before_grouped_category', 'clean_sheets_before_grouped'
]

# Normalize competition areas: keep top 20, group rest into 'Other'
for area_col in ['from_competition_competition_area', 'to_competition_competition_area']:
    top20 = df[area_col].value_counts().nlargest(20).index
    df[area_col] = df[area_col].apply(lambda x: x if x in top20 else "Other")

# Keep only valid grouped categories for scorers (e.g. '0–3', '4–7', ...)
df['scorer_before_grouped_category'] = df['scorer_before_grouped_category'].apply(
    lambda x: x if pd.notnull(x) and str(x).strip()[0].isdigit() else np.nan
)

# Define numeric features not already in other groups
numeric_features = [f for f in important_features if f not in (binary_features + log_transform_features + categorical_integer_features + categorical_features)]

# Combine all feature groups for plotting
features_to_plot = numeric_features + binary_features + categorical_integer_features + categorical_features + log_transform_features

# Set up subplot grid layout (3 columns)
cols = 3
rows = int(np.ceil(len(features_to_plot) / cols))
fig, axes = plt.subplots(rows, cols, figsize=(14, 3 * rows))
axes = axes.flatten()
fig.suptitle("Distributions of Key Features", fontsize=18)

# Loop through each feature and create appropriate plot
for i, feature in enumerate(features_to_plot):
    ax = axes[i]
    data = df[feature].dropna()

    if feature in binary_features:
        data.value_counts().sort_index().plot.bar(ax=ax, color='red', edgecolor='black')
        ax.set_xticks([0, 1])
        ax.set_xticklabels(["No (0)", "Yes (1)"])

    elif feature in categorical_integer_features:
        value_counts = data.astype(int).value_counts().reindex([1, 2, 3, 4], fill_value=0)
        value_counts.plot.bar(ax=ax, color='red', edgecolor='black')
        ax.set_xticks([0, 1, 2, 3])
        ax.set_xticklabels(["1", "2", "3", "4"])

    elif feature == 'scorer_before_grouped_category':
        data.value_counts().sort_index(
            key=lambda x: x.str.extract(r'(\d+)').astype(float).squeeze()
        ).plot.bar(ax=ax, color='red', edgecolor='black')
        ax.tick_params(axis='x', rotation=90)

    elif feature == 'clean_sheets_before_grouped':
        data.value_counts().sort_index(
            key=lambda x: x.str.extract(r'(\d+)').astype(float).squeeze()
        ).plot.bar(ax=ax, color='red', edgecolor='black')
        ax.tick_params(axis='x', rotation=0)

    elif feature in categorical_features:
        data.value_counts().sort_values(ascending=False).plot.bar(ax=ax, color='red', edgecolor='black')
        ax.tick_params(axis='x', rotation=90)

    elif feature in log_transform_features:
        data = data.replace(0, np.nan).dropna()
        np.log10(data).plot.hist(ax=ax, bins=30, color='red', edgecolor='black')
        ax.set_xlabel(f"log10({feature}) [€]")
        ax.set_title(f"Distribution of {feature} [log10]")

    else:
        data.plot.hist(ax=ax, bins=30, color='red', edgecolor='black')

    if feature not in log_transform_features:
        ax.set_title(f"Distribution of {feature}")
        ax.set_xlabel(feature)
    ax.set_ylabel("Count")

# Hide any unused axes if the number of features is not divisible by the number of columns
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()